In [1]:
from time import sleep
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from robotsmeco import Robot as rob
from tasho.OCPGenerator import OCPGenerator
import numpy as np
import casadi as cs
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env

In [2]:
robot = rob.Robot("kinova")
link_name = 7
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 1, 0, 0.6], [1, 0, 0, 0.0], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [1.0193752249977548, -0.05311582280659044, -2.815452580946695,
    1.3191046402052224, 2.8582660722530533, 1.3988994390898029,1.8226311094569714]
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 7-DoF robot: kinova


In [3]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [4]:
task_P2P.write_task_graph("P2P_Task.svg")

In [5]:
reg_jacc = task_P2P.constraint_expressions['reg_qdd_'+robot.name].change_weight(0.1)
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1))

In [6]:
task_P2P.write_task_graph("P2P_Task2.svg")

In [7]:
horizon_steps = 10
horizon_period = 3.0
OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      529
Number of nonzeros in inequality constraint Jacobian.:      217
Number of nonzeros in Lagrangian Hessian.............:      168

Total number of variables............................:      224
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [8]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

from tasho import WorldSimulator
import pybullet as p

obj = WorldSimulator.WorldSimulator()

    # Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 3, 4, 5, 6]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

pybullet build time: Dec  1 2021 18:33:04


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel Open Source Technology Center
GL_RENDERER=Mesa DRI Intel(R) UHD Graphics 630 (CFL GT2)
GL_VERSION=4.6 (Core Profile) Mesa 20.0.8
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 20.0.8
Vendor = Intel Open Source Technology Center
Renderer = Mesa DRI Intel(R) UHD Graphics 630 (CFL GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel Open Source Technology Center
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel Open 

Tunnel-following task

In [9]:
from tasho.templates.SE3_tunnel import SE3Tunnel

vel_limit = 0.5 #m/s
acc_limit = 2.0 #m/s^2

trans_tunnel_size = 0.02
rot_tunnel_size = 5/3.14159/180

link_name = 7
a_p = 0.2
z_p = 0.1
SE3_path_fun = lambda s : cs.vertcat(
                    cs.horzcat(cs.DM([[0, 1, 0], [1, 0, 0], [0, 0, -1]]), 
                    cs.vertcat(0.6+z_p*cs.sin(s*(4*np.pi)), 0.0+a_p*cs.sin(s*(2*np.pi)), 0.25+a_p*cs.sin(s*(2*np.pi))*cs.cos(s*(2*np.pi)))), 
                    cs.DM([0, 0, 0, 1]).T)
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[-1, 0, 0, 0.6], [0, 1, 0, 0.35], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [ 0.42280387,  1.56128753, -2.07387664,  1.1543891,   1.7809308,   2.03112421,
4.02677039]

tunnel_disembodiedEE = SE3Tunnel("contouring", SE3_path_fun, vel_limit, acc_limit, trans_tunnel_size, rot_tunnel_size)
tunnel_disembodiedEE.write_task_graph("tunnel_disEE.svg")

In [10]:
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

# # Using the template to create the P2P task
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)
# Removing the goal pose of P2P because not relevant for tunnel-following
task_P2P.remove_expression(goal_pose)
task_P2P.write_task_graph("P2P_Task3.svg")

In [11]:
tunnel_disembodiedEE.substitute_expression(tunnel_disembodiedEE.variables['SE3_traj_contouring'], task_P2P.expressions["pose_7_kinova"])
tunnel_disembodiedEE.write_task_graph("tunnel_disEE2.svg")

In [12]:
tunnel_task = TaskModel.compose("lemniscate", "tunnel_following", tunnel_disembodiedEE, task_P2P)
tunnel_task.write_task_graph("tunnel_following.svg")

In [13]:
horizon_steps = 30
horizon_period = 3
OCP_gen = OCPGenerator(tunnel_task, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)

OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}} 
)

OCP_gen.tc.solve_ocp()

st = OCP_gen.stage_tasks[0]
# print(OCP_gen.tc.sol_sample(st.expressions['SE3_path_contouring'].x))
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
# print(qsol)
t_grid, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)


This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1705
Number of nonzeros in inequality constraint Jacobian.:      720
Number of nonzeros in Lagrangian Hessian.............:     1320

Total number of variables............................:      736
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      505
Total number of inequality constraints...............:      720
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      600
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [14]:
obj = WorldSimulator.WorldSimulator()

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 3, 4, 5, 6]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel Open Source Technology Center
GL_RENDERER=Mesa DRI Intel(R) UHD Graphics 630 (CFL GT2)
GL_VERSION=4.6 (Core Profile) Mesa 20.0.8
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 20.0.8
Vendor = Intel Open Source Technology Center
Renderer = Mesa DRI Intel(R) UHD Graphics 630 (CFL GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel Open Source Technology Center
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel Open 

Object picking MPC

In [15]:
# Define initial conditions of the robot
q0_val = [0, -0.523598, 0, 2.51799, 0, -0.523598, -1.5708]
qd0_val = [0] * robot.ndof

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (7,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (7,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, 7, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("P2P_Task4.svg")

In [16]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_kinova'])
task_P2P.remove_terminal_constraints('rot_con_pose_7_kinova_vs_goal',
                                    'trans_con_pose_7_kinova_vs_goal')
task_P2P.write_task_graph("P2P_Task5.svg")

In [17]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_kinova']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_7_kinova_vs_goal'], 1.0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_7_kinova_vs_goal'], 1), 
                            Regularization(task_P2P.variables['qd_kinova'], 1e-4))

task_P2P.write_task_graph("P2P_Task6.svg")

In [18]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_kinova'].x
qd = pOCP.stage_tasks[0].variables['qd_kinova'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_kinova'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_kinova'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_kinova'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_kinova", "var":"qd_kinova", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_7_kinova_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
import os; os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

sh: 1: export: : bad variable name


      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  11.12ms (855.15us)   1.05ms ( 80.96us)        13
       nlp_g  |   2.42ms (186.54us) 223.75us ( 17.21us)        13
    nlp_grad  |   2.28ms (  2.28ms) 227.92us (227.92us)         1
  nlp_grad_f  |  25.13ms (  1.93ms)   2.39ms (184.00us)        13
  nlp_hess_l  | 127.92ms ( 10.66ms)  12.03ms (  1.00ms)        12
   nlp_jac_g  |   2.83ms (217.92us) 269.70us ( 20.75us)        13
       total  | 322.14ms (322.14ms)  31.21ms ( 31.21ms)         1


In [19]:
vars_db = tc.generate_MPC_component("./", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti2_p_2[12:15]-fk_T(opti2_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti2_p_2[12:15]-fk_T(opti2_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [20]:
MPC_component = MPC("kinova_obj_pickup", "./" + tc.name + ".json")

In [21]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
kinovaID = obj.add_robot(position, orientation, "kinova")

# Set environment
environment = env.Environment()

cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "../../../models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf = "../../../models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "../../../models/objects/cube_small copy.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5, 6]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetJointState(kinovaID, joint_indices, q0_val)
obj.setController(
    kinovaID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel Open Source Technology Center
GL_RENDERER=Mesa DRI Intel(R) UHD Graphics 630 (CFL GT2)
GL_VERSION=4.6 (Core Profile) Mesa 20.0.8
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 20.0.8
Vendor = Intel Open Source Technology Center
Renderer = Mesa DRI Intel(R) UHD Graphics 630 (CFL GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel Open Source Technology Center
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel Open 

In [22]:
# Execute the MPC loop
q_log = []
q_dot_log = []
predicted_pos_log = []

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(kinovaID, joint_indices)
    qd_now = obj.readJointVelocities(kinovaID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_kinova"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_kinova"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 1, 0, predicted_pos[0]]),
        cs.hcat([1, 0, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_kinova"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_kinova"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_kinova"]["val"] * t_mpc).full()
    obj.setController(
        kinovaID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
    if "termination_criteria_true" in MPC_component.event_output_port:
        break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.86ms ( 54.62us)   1.82ms ( 53.56us)        34
       nlp_g  |   1.96ms ( 57.65us) 541.30us ( 15.92us)        34
    nlp_grad  | 532.00us (532.00us) 541.50us (541.50us)         1
  nlp_grad_f  |  15.28ms (449.44us)   3.87ms (113.70us)        34
  nlp_hess_l  |  38.33ms (  1.16ms)  20.78ms (629.58us)        33
   nlp_jac_g  | 472.00us ( 13.88us) 475.03us ( 13.97us)        34
       total  | 111.46ms (111.46ms)  53.98ms ( 53.98ms)         1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  42.00us ( 42.00us)  42.61us ( 42.61us)         1
       nlp_g  |  14.00us ( 14.00us)  13.89us ( 13.89us)         1
    nlp_grad  | 206.00us (206.00us) 205.97us (205.97us)         1
  nlp_grad_f  |  99.00us ( 99.00us)  99.83us ( 99.83us)         1
   nlp_jac_g  |  11.00us ( 11.00us)  11.03us ( 

In [23]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed
